In [2]:
import pandas as pd
import numpy as np
import spacy
import gensim
import operator
import re
import string

In [3]:
df_schemes = pd.read_csv('../df.csv', encoding='mac_roman')
df_schemes.tail()

,Scheme,Description,Agency,FSC,Image,Link,Who's it for,What it gives,Scheme Type,search_booster(WL)
405,Acacia Home,A seven-storey building with a 250-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,..."
406,Bukit Batok Home for the Aged,A three-storey building with a 200-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,..."
407,Tembusu Home at Pelangi Village,A seven-storey building with a 200-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,..."
408,Thuja Home at Pelangi Village,A four-storey building with a 250-bed capacity...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,..."
409,MINDSville@Napiri,"Designed to provide professional care, accommo...",Movement for the Intellectually Disabled of Si...,NaN,https://chidnast.sirv.com/SchemesSG/minds.jpg,https://www.minds.org.sg/for-adults/residentia...,Persons with intellectual disabilities (PWIDs)...,"Housing,Accomodation,Peer support,Social Suppo...","Housing,Mental Health,Children","caregiving, caring, intellectual disability, I..."


In [4]:
#Create tag list
#df_schemes['Tags'] = [x for x in df_schemes[df_schemes.columns[5:8]].values]
#df_schemes['Tags'] = df_schemes['Tags'].apply(lambda x: [i for i in x if str(i) != "nan"])
#df_schemes

#Create concatenated column
df_schemes['Combined'] = df_schemes[['Description','search_booster(WL)']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [5]:
df_schemes

,Scheme,Description,Agency,FSC,Image,Link,Who's it for,What it gives,Scheme Type,search_booster(WL),Combined
0,Various services,Information relating to social service and ref...,@27 Family Service Centre,Yes,https://chidnast.sirv.com/SchemesSG/27fsc.jpg,https://www.27fsc.org.sg/,Families,"Counselling,Casework","Low Income,Family","social service, individuals, families, casewor...",Information relating to social service and ref...
1,Financial Assistance,Cancer treatment can place a heavy financial b...,365 Cancer Prevention Society (365CPS),NaN,https://chidnast.sirv.com/SchemesSG/365cps.jpg,https://365cps.org.sg/portfolio/financial-supp...,Cancer patients,Financial assistance for cancer treatment,"Low Income,Healthcare","See doctor, cancer, oncology",Cancer treatment can place a heavy financial b...
2,Food Assistance,A self setup group which distributes meal box ...,A Packet of Rice,NaN,https://chidnast.sirv.com/SchemesSG/apacketofr...,https://www.facebook.com/APacketOfRice/,"Low income,Need food support",Food,"Low Income,Food","Needs help to get food, meal, hungry, have not...",A self setup group which distributes meal box ...
3,Rehabilitation Centre and Respite Centre,"Provides Occupational Therapy, Physiotherapy a...",Abilities Beyond Limitations and Expectations ...,NaN,https://chidnast.sirv.com/SchemesSG/able.jpg,https://www.able-sg.org/,"Persons with disabilities (PWDs),Special needs...","Occupational therapy,Physiotherapy,Educational...","Disability,Family,Transport,Healthcare,Mental ...","rehabilitation, occupational therapy, physioth...","Provides Occupational Therapy, Physiotherapy a..."
4,Various Services,We prepare educational programs (IT Training) ...,Acronis Foundation,NaN,https://chidnast.sirv.com/SchemesSG/acronis.jpg,https://acronis.org/rehabilitation-of-inmates/,Ex-offenders,"Vocational training,Employment assistance","Prison and Ex-offenders,Work","ex-convict need job, ex-convict skills trainin...",We prepare educational programs (IT Training) ...
...,...,...,...,...,...,...,...,...,...,...,...
405,Acacia Home,A seven-storey building with a 250-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A seven-storey building with a 250-bed capacit...
406,Bukit Batok Home for the Aged,A three-storey building with a 200-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A three-storey building with a 200-bed capacit...
407,Tembusu Home at Pelangi Village,A seven-storey building with a 200-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A seven-storey building with a 200-bed capacit...
408,Thuja Home at Pelangi Village,A four-storey building with a 250-bed capacity...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A four-storey building with a 250-bed capacity...


In [6]:
from spacy.lang.en.stop_words import STOP_WORDS
import string

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

to_delete = ["alone","themselves"]
for elem in to_delete:
    stop_words.discard(elem)
stop_words.add("client")

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):

    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)

    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)

    #remove punctuations
    sentence = re.sub(r'[^\w\s]',' ',sentence)

    #creating token object
    tokens = spacy_nlp(sentence)

    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]

    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]

    #return tokens
    return tokens

In [7]:
%time df_schemes['Tags_tokenized'] = df_schemes['Combined'].map(lambda x: spacy_tokenizer(x))

df_schemes

CPU times: user 5.88 s, sys: 349 ms, total: 6.23 s
Wall time: 7.65 s


,Scheme,Description,Agency,FSC,Image,Link,Who's it for,What it gives,Scheme Type,search_booster(WL),Combined,Tags_tokenized
0,Various services,Information relating to social service and ref...,@27 Family Service Centre,Yes,https://chidnast.sirv.com/SchemesSG/27fsc.jpg,https://www.27fsc.org.sg/,Families,"Counselling,Casework","Low Income,Family","social service, individuals, families, casewor...",Information relating to social service and ref...,"[information, relate, social, service, referra..."
1,Financial Assistance,Cancer treatment can place a heavy financial b...,365 Cancer Prevention Society (365CPS),NaN,https://chidnast.sirv.com/SchemesSG/365cps.jpg,https://365cps.org.sg/portfolio/financial-supp...,Cancer patients,Financial assistance for cancer treatment,"Low Income,Healthcare","See doctor, cancer, oncology",Cancer treatment can place a heavy financial b...,"[cancer, treatment, place, heavy, financial, b..."
2,Food Assistance,A self setup group which distributes meal box ...,A Packet of Rice,NaN,https://chidnast.sirv.com/SchemesSG/apacketofr...,https://www.facebook.com/APacketOfRice/,"Low income,Need food support",Food,"Low Income,Food","Needs help to get food, meal, hungry, have not...",A self setup group which distributes meal box ...,"[self, setup, group, distribute, meal, box, lo..."
3,Rehabilitation Centre and Respite Centre,"Provides Occupational Therapy, Physiotherapy a...",Abilities Beyond Limitations and Expectations ...,NaN,https://chidnast.sirv.com/SchemesSG/able.jpg,https://www.able-sg.org/,"Persons with disabilities (PWDs),Special needs...","Occupational therapy,Physiotherapy,Educational...","Disability,Family,Transport,Healthcare,Mental ...","rehabilitation, occupational therapy, physioth...","Provides Occupational Therapy, Physiotherapy a...","[provide, occupational, therapy, physiotherapy..."
4,Various Services,We prepare educational programs (IT Training) ...,Acronis Foundation,NaN,https://chidnast.sirv.com/SchemesSG/acronis.jpg,https://acronis.org/rehabilitation-of-inmates/,Ex-offenders,"Vocational training,Employment assistance","Prison and Ex-offenders,Work","ex-convict need job, ex-convict skills trainin...",We prepare educational programs (IT Training) ...,"[prepare, educational, program, training, offe..."
...,...,...,...,...,...,...,...,...,...,...,...,...
405,Acacia Home,A seven-storey building with a 250-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A seven-storey building with a 250-bed capacit...,"[seven, storey, build, 250, bed, capacity, cat..."
406,Bukit Batok Home for the Aged,A three-storey building with a 200-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A three-storey building with a 200-bed capacit...,"[storey, build, 200, bed, capacity, caters, de..."
407,Tembusu Home at Pelangi Village,A seven-storey building with a 200-bed capacit...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A seven-storey building with a 200-bed capacit...,"[seven, storey, build, 200, bed, capacity, res..."
408,Thuja Home at Pelangi Village,A four-storey building with a 250-bed capacity...,Sathya Sai Social Service (4S),NaN,https://chidnast.sirv.com/SchemesSG/4s.jpg,https://www.4s.org.sg/Welfare-Homes-Singapore-...,Homeless,"Housing,Accomodation","Homeless,Housing","Homeless, no place to stay, runaway from home,...",A four-storey building with a 250-bed capacity...,"[storey, build, 250, bed, capacity, residentia..."


In [8]:
Description_to_match = df_schemes['Tags_tokenized']
Description_to_match[0:5]

0    [information, relate, social, service, referra...
1    [cancer, treatment, place, heavy, financial, b...
2    [self, setup, group, distribute, meal, box, lo...
3    [provide, occupational, therapy, physiotherapy...
4    [prepare, educational, program, training, offe...
Name: Tags_tokenized, dtype: object

In [9]:
#Word Cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt

series = pd.Series(np.concatenate(Description_to_match)).value_counts()[:100]
wordcloud = WordCloud(background_color='white').generate_from_frequencies(series)

plt.figure(figsize=(15,15), facecolor = None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

ModuleNotFoundError: No module named 'wordcloud'

In [10]:
#build word dictionary

from gensim import corpora

#creating term dictionary
%time dictionary = corpora.Dictionary(Description_to_match)

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
#NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=4, no_above=0.2)

#list of few which which can be further removed
stoplist = set('hello and if this can would should could tell ask come go')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)
dictionary.save('dictionary')

CPU times: user 23.1 ms, sys: 1.22 ms, total: 24.3 ms
Wall time: 29.3 ms


In [11]:
#print top 50 items from the dictionary with their unique token-id
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 50]]
print (dict_tokens)

[[['appropriate', 0], ['casework', 1], ['counsel', 2], ['counselling', 3], ['family', 4], ['groupwork', 5], ['individual', 6], ['information', 7], ['referral', 8], ['relate', 9], ['service', 10], ['social', 11], ['therapeutic', 12], ['365cps', 13], ['aid', 14], ['burden', 15], ['cancer', 16], ['daily', 17], ['doctor', 18], ['educate', 19], ['emotional', 20], ['financial', 21], ['find', 22], ['health', 23], ['heavy', 24], ['help', 25], ['immunity', 26], ['improve', 27], ['income', 28], ['low', 29], ['meet', 30], ['member', 31], ['need', 32], ['oncology', 33], ['oneís', 34], ['patient', 35], ['place', 36], ['provide', 37], ['support', 38], ['treatment', 39], ['box', 40], ['bukit', 41], ['child', 42], ['citizen', 43], ['day', 44], ['distribute', 45], ['eat', 46], ['flat', 47], ['food', 48], ['group', 49], ['hungry', 50]]]


In [12]:
#Bag of words
corpus = [dictionary.doc2bow(desc) for desc in Description_to_match]
word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:3]]
print(word_frequencies)

[[('appropriate', 1), ('casework', 2), ('counsel', 1), ('counselling', 1), ('family', 1), ('groupwork', 1), ('individual', 1), ('information', 1), ('referral', 1), ('relate', 1), ('service', 3), ('social', 2), ('therapeutic', 1)], [('family', 1), ('365cps', 1), ('aid', 1), ('burden', 1), ('cancer', 3), ('daily', 1), ('doctor', 1), ('educate', 1), ('emotional', 1), ('financial', 1), ('find', 1), ('health', 1), ('heavy', 1), ('help', 1), ('immunity', 1), ('improve', 1), ('income', 1), ('low', 1), ('meet', 1), ('member', 1), ('need', 1), ('oncology', 1), ('oneís', 1), ('patient', 2), ('place', 2), ('provide', 1), ('support', 1), ('treatment', 1)], [('help', 1), ('income', 1), ('low', 1), ('need', 2), ('box', 1), ('bukit', 1), ('child', 1), ('citizen', 1), ('day', 1), ('distribute', 1), ('eat', 1), ('flat', 1), ('food', 2), ('group', 1), ('hungry', 1), ('meal', 2), ('merah', 1), ('near', 1), ('needy', 1), ('operate', 1), ('region', 1), ('rent', 1), ('reside', 1), ('resident', 1), ('room', 

In [13]:
#TF IDF and LSI
%time schemes_tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
schemes_tfidf_model.save("tfidf.model")
%time schemes_lsi_model = gensim.models.LsiModel(schemes_tfidf_model[corpus], id2word=dictionary, num_topics=300)
schemes_lsi_model.save("lsi.model")

CPU times: user 8.58 ms, sys: 349 μs, total: 8.93 ms
Wall time: 11.4 ms
CPU times: user 554 ms, sys: 63.6 ms, total: 617 ms
Wall time: 441 ms


In [14]:
%time gensim.corpora.MmCorpus.serialize('schemes_tfidf_model_mm', schemes_tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('schemes_lsi_model_mm',schemes_lsi_model[schemes_tfidf_model[corpus]])

CPU times: user 114 ms, sys: 7.07 ms, total: 121 ms
Wall time: 159 ms
CPU times: user 382 ms, sys: 17 ms, total: 399 ms
Wall time: 465 ms


In [15]:
#Load indexed corpus
schemes_tfidf_corpus = gensim.corpora.MmCorpus('schemes_tfidf_model_mm')
schemes_lsi_corpus = gensim.corpora.MmCorpus('schemes_lsi_model_mm')

print(schemes_tfidf_corpus)
print(schemes_lsi_corpus)

MmCorpus(410 documents, 2774 features, 13333 non-zero entries)
MmCorpus(410 documents, 300 features, 123000 non-zero entries)


In [16]:
from gensim.similarities import MatrixSimilarity

%time schemes_index = MatrixSimilarity(schemes_lsi_corpus, num_features = schemes_lsi_corpus.num_terms)

CPU times: user 121 ms, sys: 5.99 ms, total: 127 ms
Wall time: 176 ms


In [17]:
#Search similarity

from operator import itemgetter

def search_similar_schemes(search_term):
    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = schemes_tfidf_model[query_bow]
    query_lsi = schemes_lsi_model[query_tfidf]

    schemes_index.num_best = 50

    schemes_list = schemes_index[query_lsi]

    schemes_list.sort(key=itemgetter(1), reverse=True)
    schemes_names = []

    for j, scheme in enumerate(schemes_list):

        schemes_names.append (
            {
                'Relevance': round((scheme[1] * 100),2),
                'Scheme Title': df_schemes['Scheme'][scheme[0]],
                'Description': df_schemes['Description'][scheme[0]]
            }

        )
        if j == (schemes_index.num_best-1):
            break

    return pd.DataFrame(schemes_names, columns=['Relevance','Scheme Title','Description'])

In [18]:
search_similar_schemes('pregnant teen')

,Relevance,Scheme Title,Description
0,78.75,HCSA Dayspring SPIN,SPIN empowers single parents to make informed ...
1,63.69,Pregnancy Crisis and Support,"For emotional support, guidance, help and refe..."
2,55.09,Babes - A Helping Hand for Pregnant Teens,Staff will discuss the various options availab...
3,36.80,Safe Place,"Safe Place provides timely, non-judgmental and..."
4,28.24,Information,"Information on various issues, including contr..."
5,8.30,"Evergreen Place, Home@Hong San (SASCO)",A shelter or community home provides:\nFinanci...
6,7.65,Casework and Counselling,"PPIS serves clients who face marital issues, f..."
7,3.97,Sexual Assualt Care Centre,"The Care Centre provides safe, free and confid..."
8,3.22,Family services,Casework management and counselling including ...
9,2.96,PSALT Care Peer Support Groups,"PSALT Care Ltd (pronounced as ëSalt-careí), is..."


In [19]:
import flask
print(flask.__version__)

3.0.3


/var/folders/9s/ms6xqds12571tgww7f0815kh0000gn/T/ipykernel_71713/3294412508.py:2: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print(flask.__version__)


In [20]:
import pickle
print(pickle.format_version)

4.0
